### Load Dataset

In [65]:
# Import Dependencies# Impor 
import pandas as pd
import numpy as np
from collections import defaultdict

In [66]:
df = pd.read_excel('dataset.xlsx')

In [67]:
df.head()

,Disease,Count of Disease Occurrence,Symptom
0,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0008031_pain chest
1,NaN,NaN,UMLS:C0392680_shortness of breath
2,NaN,NaN,UMLS:C0012833_dizziness
3,NaN,NaN,UMLS:C0004093_asthenia
4,NaN,NaN,UMLS:C0085639_fall


In [68]:
data = df.fillna(method='ffill')

In [69]:
data.head()

,Disease,Count of Disease Occurrence,Symptom
0,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0008031_pain chest
1,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0392680_shortness of breath
2,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0012833_dizziness
3,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0004093_asthenia
4,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0085639_fall


### Preprocessing

In [70]:
#Process Disease and Symptom Names
def process_data(data):
    data_list = []
    data_name = data.replace('^','_').split('_')
    n = 1
    for names in data_name:
        if (n % 2 == 0):
            data_list.append(names)
        n += 1
    return data_list

In [71]:
data = data.drop(['Count of Disease Occurrence'], axis=1)

In [72]:
disease_list = []
disease_symptom_dict = defaultdict(list)
disease_symptom_count = {}
count = 0

for idx, row in data.iterrows():
    
    # Get the Disease Names
    if (row['Disease'] !="\xc2\xa0") and (row['Disease'] != ""):
        disease = row['Disease']
        disease_list = process_data(data=disease)

    # Get the Symptoms Corresponding to Diseases
    if (row['Symptom'] !="\xc2\xa0") and (row['Symptom'] != ""):
        symptom = row['Symptom']
        symptom_list = process_data(data=symptom)
        for d in disease_list:
            for s in symptom_list:
                disease_symptom_dict[d].append(s)

In [73]:
disease_symptom_dict

defaultdict(list,
            {"Alzheimer's disease": ['drool',
              'agitation',
              'nightmare',
              'rhonchus',
              'consciousness clear',
              'pin-point pupils',
              'bedridden',
              'bedridden',
              'frail',
              'tremor resting',
              'hyperkalemia',
              'facial paresis',
              'groggy',
              'muscle twitch',
              'wheelchair bound',
              'tremor',
              'cough',
              'fever'],
             'HIV': ['fever',
              'night sweat',
              'spontaneous rupture of membranes',
              'cough',
              '',
              'decreased body weight',
              'chill',
              'diarrhea',
              'pleuritic pain',
              'patient non compliance',
              'tachypnea',
              'productive cough',
              'muscle hypotonia',
              'hypotonic',
              'feeling

### model training

In [74]:
from sklearn.feature_extraction.text import CountVectorizer

In [75]:
x = []
y = []
for i in disease_symptom_dict:
    y.append(i)
    x.append(disease_symptom_dict[i])

In [76]:
# temp contains all possible symptoms with no duplications
temp = []
for i in x:
    for j in i:
        temp.append(j)

from collections import OrderedDict
temp = list(OrderedDict((x, True) for x in temp).keys())

In [77]:
import random
from math import floor

for i in range(len(y)):
   for j in range(150):
       y.append(y[i])
       x.append(random.sample(x[i],floor(len(x[i])/3)))

In [78]:
x_ = []
for i in x:
   x_.append(' '.join(i))
x_

['pain food intolerance numbness of hand general discomfort drowsiness asthenia nonsmoker non-productive cough polydypsia stiffness unsteady gait',
 'polyuria vomiting nausea pain abdominal milky feeling strange gurgle nervousness abdominal tenderness regurgitates after swallowing vision blurred urinary hesitation diarrhea seizure aura',
 'hemiplegia snore unresponsiveness productive cough dyspnea tachypnea hyperkalemia hypokinesia sinus rhythm general unsteadiness bruit consciousness clear shortness of breath lesion redness',
 'pain abdominal fatigability prodrome vomiting nausea myalgia hyponatremia sore to touch general discomfort dyspnea on exertion asterixis guaiac positive numbness of hand headache photophobia',
 'pain abdominal paraparesis nausea vomiting polymyalgia out of breath pain chest hemiplegia gurgle hunger apyrexial nervousness',
 'fever chill flushing unresponsiveness indifferent mood urinoma vomiting distended abdomen hypoalbuminemia pustule prostatism diarrhea abdom

In [79]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x_, y, test_size = 0.2)   

In [80]:
len(y_test)

4500

In [81]:
# import numpy as np
# import re


# temp = sorted(list(set(temp)))

# def extract_words(sentence):
#     ignore_words = ['a']
#     words = re.sub("[^\w]", " ",  sentence).split() #nltk.word_tokenize(sentence)
#     words_cleaned = [w.lower() for w in words if w not in ignore_words]
#     return words_cleaned
    
# def bagofwords(sentence, words):
#     sentence_words = extract_words(sentence)
#     # frequency word count
#     bag = [0] * len(temp)
#     for sw in sentence_words:
#         for i,word in enumerate(words):
#             if word == sw: 
#                 bag[i] += 1
                
#     return np.array(bag)

# def convert_using_bagofwords(sentences):
#     result = []
#     for sen in sentences:
#         result.append(bagofwords(sen, temp))

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
from numpy import array

count_vector = CountVectorizer()

count_vector.fit(temp)
len(count_vector.get_feature_names())


539

In [88]:
#Fit the training data and then return the matrix
training_data = count_vector.fit_transform(x_train)

#Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(x_test)

In [89]:
# training_data = []
# testing_data = []

# training_data = convert_using_bagofwords(x_train)
# testing_data = convert_using_bagofwords(x_test)

In [90]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [91]:
predictions = naive_bayes.predict(testing_data)

In [92]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
#print('Precision score: ', format(precision_score(y_test, predictions, average='micro')))
#print('Recall score: ', format(recall_score(y_test, predictions,average='micro')))
#print('F1 score: ', format(f1_score(y_test, predictions, average='micro')))

Accuracy score:  0.8666666666666667


### Trials

In [ ]:
pred = naive_bayes.predict()